In [4]:
%load_ext autotime

In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

In [2]:
import magellan as mg

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened
DEBUG:Cloud:Deleting 453.lock (8.3433292869 days old)
DEBUG:Cloud:Deleting 453.log (8.3433292869 days old)
DEBUG:Cloud:Deleting 545.lock (8.33976447208 days old)
DEBUG:Cloud:Deleting 545.log (8.33976447208 days old)


In [3]:
A = mg.read_csv('AMAZON_bkp_1000.csv',key='ID')

In [5]:
#A

time: 671 µs


In [6]:
B = mg.read_csv('BARNES_bkp_1000.csv',key='ID')

time: 14 ms


In [7]:
ob = mg.OverlapBlocker()

time: 1.53 ms


In [8]:
C = ob.block_tables(A,B,'Publisher','Publisher',overlap_size=1,l_output_attrs=['Title','Author','ISBN_13','Publisher'],r_output_attrs=['Publisher','Title','Author','ISBN_13'])

0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 4.01 s



Total time elapsed: 3.044 sec


In [9]:
#C

time: 652 µs


In [10]:
rb = mg.RuleBasedBlocker()

time: 1.13 ms


In [11]:
block_t = mg.get_single_arg_tokenizers(q=[3],dlm_char=['# ',' '])

time: 1.04 ms


In [12]:
block_t

{'dlm_dc0': <function magellan.feature.tokenizers.tok_delim>,
 'dlm_dc1': <function magellan.feature.tokenizers.tok_delim>,
 'qgm_3': <function magellan.feature.tokenizers.tok_qgram>}

time: 7.8 ms


In [13]:
block_s = mg.get_sim_funs()

time: 934 µs


In [14]:
block_s

{'abs_norm': <function magellan.feature.simfunctions.abs_norm>,
 'cosine': <function magellan.feature.simfunctions.cosine>,
 'exact_match': <function magellan.feature.simfunctions.exact_match>,
 'jaccard': <function magellan.feature.simfunctions.jaccard>,
 'jaro': <function magellan.feature.simfunctions.jaro>,
 'jaro_winkler': <function magellan.feature.simfunctions.jaro_winkler>,
 'lev': <function magellan.feature.simfunctions.lev>,
 'monge_elkan': <function magellan.feature.simfunctions.monge_elkan>,
 'needleman_wunsch': <function magellan.feature.simfunctions.needleman_wunsch>,
 'rel_diff': <function magellan.feature.simfunctions.rel_diff>,
 'smith_waterman': <function magellan.feature.simfunctions.smith_waterman>,
 'smith_waterman_gotoh': <function magellan.feature.simfunctions.smith_waterman_gotoh>,
 'soundex': <function magellan.feature.simfunctions.soundex>}

time: 2.73 ms


In [15]:
A = mg.MTable(A.applymap(str), key='ID') # Pradap

time: 17.9 ms


In [16]:
A.dtypes

ID                    object
Title                 object
Author                object
Price                 object
Edition               object
ASIN                  object
ISBN_13               object
ISBN_10               object
Paperback             object
Series                object
Publisher_dummy       object
Publisher             object
Publication_Date      object
Sold_by               object
Language              object
Product Dimensions    object
Shipping Weight       object
dtype: object

time: 5.03 ms


In [17]:
B = mg.MTable(B.applymap(str), key='ID')

time: 12.9 ms


In [18]:
B.dtypes

ID                    object
Title                 object
Author                object
Hardcover             object
Paperback             object
NOOK_Book             object
Audiobook             object
ISBN_13_DUMMY         object
ISBN_13               object
Series                object
Publisher             object
Publication_Date      object
Sales_rank            object
Pages                 object
Product_dimensions    object
dtype: object

time: 3.99 ms


In [27]:
atypes1 = mg.get_attr_types(A)
atypes1['Author'] = 'str_bt_1w_5w' # Pradap - set attribute type for Author

time: 39.8 ms


In [20]:
#atypes1

time: 774 µs


In [25]:
atypes2 = mg.get_attr_types(B)
atypes2['Author']

'str_bt_1w_5w'

time: 38.8 ms


In [22]:
#atypes2

time: 724 µs


In [23]:
block_c = mg.get_attr_corres(A,B)

time: 1.02 ms


In [28]:
block_f = mg.get_features(A, B, atypes1, atypes2, block_c, block_t, block_s) # Pradap - Now this should contain Author features

time: 7.43 ms


In [30]:
block_f.head(2)

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,ID_ID_lev,ID,ID,None,None,lev,<function ID_ID_lev at 0x115883848>,from magellan.feature.simfunctions import *\nf...
1,ID_ID_jar,ID,ID,None,None,jaro,<function ID_ID_jar at 0x1158835f0>,from magellan.feature.simfunctions import *\nf...


time: 12.3 ms


In [31]:
#mg.init_jvm("C:\Program Files\Java\jdk1.8.0_60\jre\\bin\server\jvm.dll")
mg.init_jvm()

True

time: 300 ms


In [46]:
rb = mg.RuleBasedBlocker()

time: 1.16 ms


In [47]:
# rb.add_rule(['Publisher_Publisher_lev > 0.1'], block_f) 
# You should do like this:
rb.add_rule(['Publisher_Publisher_lev(ltuple, rtuple) > 0.1'], block_f)

True

time: 2.77 ms


In [48]:
R = rb.block_tables(A, B, l_output_attrs=['Publisher'], r_output_attrs=['Publisher'])


0%                          100%
[##############################] | ETA[sec]: 0.000 

time: 5min 59s



Total time elapsed: 357.164 sec


In [50]:
R.head(4)

,_id,ltable.ID,rtable.ID,ltable.Publisher,rtable.Publisher
0,0,1,8.html,Tian Xia Wen Hua,Lulu.com
1,1,1,13.html,Tian Xia Wen Hua,Masked Fox Productions
2,2,1,28.html,Tian Xia Wen Hua,Pearson Education
3,3,1,29.html,Tian Xia Wen Hua,NOLO


time: 10.2 ms


ID


0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.834 sec


,_id,ltable.ID,rtable.ID,ltable.Title,ltable.Author,ltable.ISBN_13,ltable.Publisher,rtable.Publisher,rtable.Title,rtable.Author,rtable.ISBN_13
